In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
from tqdm.notebook import tqdm
import time
from IPython.display import clear_output

import ee, geemap

In [2]:
%load_ext autoreload
%autoreload 2
import sys  
sys.path.insert(0, '../../lib/')

In [3]:
%reload_ext autoreload
import sand_classification as sc
import quality_mosaic as qm

# Part I: Get the locations from Google Drive

In [4]:
sheet_id = '13nF_pJ02Bd70cDJamuKbvZIkIdJ-kOI4O3Cx9K7Wzos'
# sheet_id = '1XqOtPkiE_Q0dfGSoyxrH730RkwrTczcRbDeJJpqRByQ'
sheet_name = 'mozambique'
# sheet_name = 'sample_1'
drive_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
drive_url

'https://docs.google.com/spreadsheets/d/13nF_pJ02Bd70cDJamuKbvZIkIdJ-kOI4O3Cx9K7Wzos/gviz/tq?tqx=out:csv&sheet=mozambique'

In [5]:
#Read in a google sheets with the ground truth data from string drive_loc
df = pd.read_csv(drive_url)
df

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes
0,moz-0,-25.571043,32.252410,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10747CM applied for
1,moz-1,-24.505472,33.012067,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10128CM Granted (2020-02-03 to 2030-02...
2,moz-2,-24.647689,33.283443,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 6528CM Granted (2013-08-22 to 2026-01-19)
3,moz-3,-24.633682,33.283166,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8996CM Granted (2018-04-01 to 2028-04-01)
4,moz-4,-25.563075,32.209854,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 3507CM Granted (2015-03-31 to 2017-03-31)
5,moz-5,-25.353037,32.327312,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8348C Applied (2016-09-61)


In [6]:
#look for any NaN values in the column 'date' or 'Longitude' or 'Latitude' in df_merged and drop them
df = df.dropna(subset=['Date', 'Longitude', 'Latitude']).reset_index(drop=True)
df

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes
0,moz-0,-25.571043,32.252410,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10747CM applied for
1,moz-1,-24.505472,33.012067,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10128CM Granted (2020-02-03 to 2030-02...
2,moz-2,-24.647689,33.283443,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 6528CM Granted (2013-08-22 to 2026-01-19)
3,moz-3,-24.633682,33.283166,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8996CM Granted (2018-04-01 to 2028-04-01)
4,moz-4,-25.563075,32.209854,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 3507CM Granted (2015-03-31 to 2017-03-31)
5,moz-5,-25.353037,32.327312,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8348C Applied (2016-09-61)


In [7]:
assert(len(df[df['Date'].isna() | df['Longitude'].isna() | df['Latitude'].isna()]) == 0)

# Part II: Select subset for training

Here we iterate over the df and iteratively pick and adjust the locations we want to keep

In [8]:
df.columns

Index(['ID', 'Latitude', 'Longitude', 'Class', 'Date', 'Site', 'Source',
       'Country', 'Notes'],
      dtype='object')

In [9]:
ee.Initialize()
# ee.Authenticate()

In [10]:
output = df.copy()
output

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes
0,moz-0,-25.571043,32.252410,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10747CM applied for
1,moz-1,-24.505472,33.012067,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10128CM Granted (2020-02-03 to 2030-02...
2,moz-2,-24.647689,33.283443,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 6528CM Granted (2013-08-22 to 2026-01-19)
3,moz-3,-24.633682,33.283166,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8996CM Granted (2018-04-01 to 2028-04-01)
4,moz-4,-25.563075,32.209854,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 3507CM Granted (2015-03-31 to 2017-03-31)
5,moz-5,-25.353037,32.327312,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8348C Applied (2016-09-61)


In [12]:
%reload_ext autoreload
import sand_classification as sc
import quality_mosaic as qm

In [14]:
INDEX = 0
sampling_buffer_m = 5
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Manual Loop

Keep running the following 2 cells until you're done with the manual sampling.


In [29]:
s1_s2, sample = qm.get_s1s2_data(output, Map, INDEX, display_smap=False, mosaic_method='qm', sampling_buffer_m=sampling_buffer_m, median_samples=5,roi_buffer_m=400, obia=True)

Index:  5  ID:  moz-5 Class:  sand  Country:  Mozambique  Site:  fluvial
Search window from 2022-01-22 to 2023-01-22


## Use this immediately after the map is refreshed to either select or discard the sample

INDEX will auto update on calling this function

In [30]:
output, INDEX = qm.get_training_sample(output, s1_s2, sample, Map, INDEX, display_clusters=False, obia=True)
output.iloc[INDEX-1]

New marker accepted
Kept Observation


ID                                                         moz-5
Latitude                                              -25.353174
Longitude                                              32.327378
Class                                                       sand
Date                                                  2022-07-22
Site                                                     fluvial
Source              https://portals.landfolio.com/mozambique/en/
Country                                               Mozambique
Notes                         License 8348C Applied (2016-09-61)
B2_mean                                              1280.482788
B3_mean                                              1855.517212
B4_mean                                                   2596.0
B8_mean                                              3560.528809
B8A_mean                                             3573.143799
B11_mean                                             4922.695312
B12_mean                 

## Final Output

In [31]:
output

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,B2_mean,...,B8A_mean,B11_mean,B12_mean,VV_mean,VH_mean,mTGSI_mean,BSI_mean,NDWI_mean,keep,location_tweaked
0,moz-0,-25.569669,32.252534,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10747CM applied for,1449.594238,...,3496.753662,4961.000000,4555.420410,-17.552206,-26.786810,0.173039,0.228689,-0.231081,True,True
1,moz-1,-24.505472,33.012067,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10128CM Granted (2020-02-03 to 2030-02...,1325.271362,...,3324.782959,4737.139648,4739.217285,-16.774797,-27.074543,0.201724,0.234520,-0.251508,True,False
2,moz-2,-24.647689,33.283443,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 6528CM Granted (2013-08-22 to 2026-01-19),1013.804871,...,2835.146240,4133.804688,4107.097656,-16.950619,-29.517706,0.211118,0.245005,-0.296115,True,False
3,moz-3,-24.633682,33.283166,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8996CM Granted (2018-04-01 to 2028-04-01),911.324524,...,2604.834473,3819.920410,3836.006592,-17.742294,-31.180273,0.223607,0.255296,-0.298279,True,False
4,moz-4,-25.563075,32.209854,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 3507CM Granted (2015-03-31 to 2017-03-31),1539.061279,...,3684.836670,5094.271973,4696.789062,-15.571375,-25.351002,0.164142,0.216804,-0.240797,True,False
5,moz-5,-25.353174,32.327378,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8348C Applied (2016-09-61),1280.482788,...,3573.143799,4922.695312,4636.310547,-14.956991,-22.532507,0.171499,0.216569,-0.315728,True,True


In [120]:
output.to_csv('../../data/labels/mozambique-gt.csv')
#save fm_df to a pickle file
output.to_pickle('../../data/labels/mozambique-gt.pkl')